<a href="https://colab.research.google.com/github/kipsangmarion/jkuat-student-admission-simulation/blob/main/JKUAT_Student_Admission_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Problem Formulation

# 2. Objectives and Overall Project Plan

## 2.1 Objectives/ Questions to be answered by system

* What is the average waiting time for students in the queue?
* What is the average amount of time the server is idle?
* What is the average service time for a student?
* What is the average inter-arrival time?
* What is the average amount of time a student spends in the system?
* How can we optimize the queueing system to attend to students more efficiently?

## 2.2 Project Plan

### 2.2.1//State the alternativen systems and methods of evaluating them//

### 2.2.2 Project team:

Shirley Mbeyu Ddaiddo

Ashley Mbeke Muoki

Ceasar Njuguna Ngunu

Marion Cherotich Kipsang

Allan Erissat Okedo

### 2.2.3 Cost of the study

//maybe a water budget//

### 2.2.4 //No of days rewuired to accomplisheach phase




# 3. Model Conceptualization

We consider a system for admitting students to Jomo Kenyatta University of agriculture and Technology. We then consider a sistuation with two servers attending to a single queue of students, and the number of students to be served is 40,000 students.

We collect data on student arrival times and service times. We the consider a 1000 hours period of time. From that data, and for each student, we determine:
* the inter-arrival times:
    - This is the difference between the arrival times of the current students in the queue and the student who arrived right before them in the queue.
    - The inter-arrival time for the first student in the queue is zero.
* time service begins:
    - Time service begins is dependent on the arrival time of the current student as compared to the time service ends for the previous student.
    - If a student arrives before the previous student has been finished being served, their service begins when the previous student finishes being served.
    - If a student arrives when the previous student has already been served, their service begins as soon as they arrive.
* time service ends:
    - Time that service ends for a student is equal to the sum of their arrival time and the time taken to serve them.
* the queue wait times:
    - Time that a student spends queueing depends on their arrival time as compared to the time service ends for the previous student.
    - If a student arrives before the previous student has been finished being served, their time in the queue is the difference between the time service ends for the previous student and the current students arrival time.
    - If a student arrives when the previous student has already been served, they do not spend any time queueing.
* time in the system:
    - The time a student spends in the system is the sum of the time they spent queueing and the time spend being served.
* server idle times:
    - Server idle times depends on the current students arrival time as compared to the time service ends for the previous student
    - If a student arrives before another student has been finished being served, the server has no idle time since there is a student in the queue.
    - If a student arrives when there is no one being served, the server is idle until the another student arrives.

From the calculated data, we can determine the averages required, the effectiveness of the system, and how we can optimize the system.

# 4. Data Collection

Data for the system is computer generated using pseudo-random numbers.

* Arrival Time:

Arrival time for students in minutes is generated using a function that generated pseudo-random numbers that increase. The fuction generates 40,000 arrival times that span 1000 hours/ 60,000 minutes.

* Service Time:

Service time for students in minutes is generated using a sin function that outputs pseudo-random values that are then multiplied by 10.

# 5. Model Translation

The above model is translated into python code.

## 5.1 Adding dependecies
The system requires:
* pandas, and numpy to compute calculations on data frames and arrays
* random for the generation of pseudo-random numbers
* math for access to the sine function

In [1]:
import pandas as pd
import numpy as np
import random
import math
from math import sin

Initializing the number of servers

In [26]:
n_servers = 2

## 5.2 Generating student service times

In [27]:
service_times= []

for i in range(40000):
  num = random.random()
  time = round((10*(sin(num)+1)/2), 2)
  service_times.append(time)

len(service_times)

40000

## 5.3 Generating student arrival times

In [28]:
arrival_times = []

arrival_time = 10*random.random()
arrival_times.append(round(arrival_time, 2))

for i in range(39999):
    arrival_time = 10*random.random() * ((60000-arrival_times[i])/60000) + arrival_times[i]
    arrival_times.append(round(arrival_time, 2))

for i in range(39999):
     arrival_times[i]

len(arrival_times)

40000

## 5.4 Calculating inter-arrival times

In [29]:
inter_arrival_times = [0]
for i in range(1, 40000):
    inter_arrival_time = round(arrival_times[i] - arrival_times[i - 1], 2)
    inter_arrival_times.append(inter_arrival_time)

len(inter_arrival_times)

40000

## 5.5 Calculations for the following data:

* When service begins
* Time spent queueing
* When service ends
* Time spent in the system
* Server idle time

In [30]:
service_begin_times = [[arrival_times[0]] for _ in range(n_servers)]
queue_wait_times = [[0.00] for _ in range(n_servers)]
service_end_times = [[service_begin_times[0][0] + service_times[0]] for _ in range(n_servers)]
times_in_system = [[service_times[0]] for _ in range(n_servers)]
server_idle_times = [[0] for _ in range(n_servers)]

for i in range(1, 40000):
    # Find the server that ends service earliest
    server_id = np.argmin([service_end_times[j][-1] for j in range(n_servers)])

    if arrival_times[i] > service_end_times[server_id][-1]:
        service_begin_time = arrival_times[i]
        queue_wait_time = 0.00
        server_idle_time = arrival_times[i] - service_end_times[server_id][-1]
    else:
        service_begin_time = service_end_times[server_id][-1]
        queue_wait_time = service_end_times[server_id][-1] - arrival_times[i]
        server_idle_time = 0.00

    service_end_time = round(service_times[i] + service_begin_time, 2)
    time_in_system = round(queue_wait_time + service_times[i], 2)

    service_begin_times[server_id].append(service_begin_time)
    queue_wait_times[server_id].append(queue_wait_time)
    service_end_times[server_id].append(service_end_time)
    times_in_system[server_id].append(time_in_system)
    server_idle_times[server_id].append(server_idle_time)

## 5.6 Adding all the data to a dataframe

In [31]:
# Creating a separate DataFrame for each server
for i in range(n_servers):
    ss = pd.DataFrame(
        {'service_time': service_times[:len(service_begin_times[i])],
         'arrival_time': arrival_times[:len(service_begin_times[i])],
         'inter_arrival_time': inter_arrival_times[:len(service_begin_times[i])],
         'time_service_begins': service_begin_times[i],
         'queue_wait_time': queue_wait_times[i],
         'time_service_ends': service_end_times[i],
         'time_in_system': times_in_system[i],
         'server_idle_time': server_idle_times[i]
        })

## 5.9 Converting the data to a csv file

In [33]:
ss.to_csv(f'admission_data_server_{i}.csv')

## 5.10 Calculating the results

In [34]:
average_waiting_time = ss['queue_wait_time'].mean()
average_idle_server_time = ss['server_idle_time'].mean()
average_service_time = ss['service_time'].mean()
average_inter_arrival_time = ss['inter_arrival_time'].mean()
average_time_in_system = ss['time_in_system'].mean()


In [35]:
print(f"The average waiting time is: {round(average_waiting_time,2)}")
print(f"The average idle server time is: {average_idle_server_time}")
print(f"The average service time is: {round(average_service_time, 2)}")
print(f"The average inter arrival time is:{round(average_inter_arrival_time, 2)}")
print(f"The average time a customer spent in the system: {round(average_time_in_system, 2)}")

The average waiting time is: 32752.0
The average idle server time is: 0.12226965955106803
The average service time is: 7.3
The average inter arrival time is:2.43
The average time a customer spent in the system: 32759.3


# 6. Model Verification

# 7. Model Validation

# 8. Experimental design